## Installation

Installs the necessary dependencies & import them for use

In [104]:
pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [105]:
import csv
import pandas as pd

## Data Preprocessing:

When we perform our analysis, we want to see the trend of each price based on the `date` field in both data. However, the `date` field in `bitcoin.csv` and `gold.csv` doesn't match up. In addition, we want the price to be processable as numbers but `bitcoin.csv` is saving `price` as string. Therefore, our primary task in preprocessing involves 2 steps:

1. Standardize `date` to `MM/YYYY` format
2. Cast `price` in `bitcoin.csv` from string format into number format

In [114]:
# read our csv as dataframe
bitcoin_df = pd.read_csv("commodity-data/bitcoin.csv")
gold_df = pd.read_csv("commodity-data/gold.csv")

In [115]:
# helper function to change date formatting as MM/YYYY
def convert_date_format(data_df):
  if "date" not in data_df.columns:
    raise ValueError("input data does not contain `date` column")

  # convert to datetime format
  data_df["date"] = pd.to_datetime(
    data_df["date"],
    format="%m/%d/%Y",
    errors="coerce"
  )
  
  # sort based on date
  data_df.sort_values(by="date", ascending=True, inplace=True)
  # change to MM/YYYY format
  
  data_df["date"] = data_df["date"].dt.strftime("%m/%Y")


# helper function to save our cleaned data into cleaned-data folder
def save_cleaned_data(data_df, filepath):
  data_df = data_df.set_index('date')
  data_df.to_csv(filepath)
  
  
# 1. standardize date format
convert_date_format(bitcoin_df)
convert_date_format(gold_df)


# optionally, we sort only save date & price
bitcoin_df = bitcoin_df[["date", "price"]]

# 2. cast the price column of bitcoin's data from string to number
bitcoin_df["price"] = bitcoin_df["price"].str.replace(",", "", regex=True)
bitcoin_df["price"] = pd.to_numeric(bitcoin_df["price"], errors="coerce")


In [116]:
# save the preprocessed data into cleaned-data folder
save_cleaned_data(gold_df, filepath="cleaned-data/gold_cleaned.csv")
save_cleaned_data(bitcoin_df, filepath="cleaned-data/bitcoin_cleaned.csv")

## Visualizing price

Now that we have standardized the date of our data, we can try to visualize how their prices (monthly average) changed overtime

In [125]:
bitcoin_df = pd.read_csv("cleaned-data/bitcoin_cleaned.csv", index_col=False)
gold_df = pd.read_csv("cleaned-data/gold_cleaned.csv", index_col=False)

In [132]:
aggregate_df = pd.merge(
  bitcoin_df[['date', 'price']],
  gold_df[['date', 'price']],
  on='date',
  suffixes=('_bitcoin', '_gold'),
  how="outer"
)

aggregate_df.fillna(0, inplace=True)

aggregate_df

,date,price_bitcoin,price_gold
0,01/1978,0.0,207.8
1,01/1979,0.0,455.1
2,01/1980,0.0,594.9
3,01/1981,0.0,410.1
4,01/1982,0.0,444.3
...,...,...,...
570,12/2020,28949.4,1856.0
571,12/2021,46219.5,1786.7
572,12/2022,16537.4,1796.7
573,12/2023,42272.5,2029.3
